### Analysis description
Create nifti mask files from ROIs drawing based on Fullscreen retinotopy

In [1]:
# Imports
import os
import nibabel as nb
import ipdb
import warnings
import cortex
from pathlib import Path
warnings.filterwarnings('ignore')

# Define parameters
n_subjects= 2 + 1
num = []
for t in range(1, n_subjects):
    num.append(str(t).zfill(3))
subjects = ['sub-{n}'.format(n=n) for n in num]
rois = ['V1','V2','V3','V3AB','LO','VO','hMT+','iIPS','sIPS','iPCS','sPCS','mPCS']

# Define folders
main_dir = '/home/mchevillard/disks/meso_shared/gaze_exp/'
pp_dir = "{}/derivatives/pp_data".format(main_dir)
pycortex_dir = "{}/cortex".format(pp_dir)
cortical_mask = 'cortical'
xfm_name = 'prf.fmriprep'

In [2]:
# Write pycortex config file
pycortex_db = "{}/db/".format(pycortex_dir)
pycortex_cm = "{}/colormaps/".format(pycortex_dir)
pycortex_config_file  = cortex.options.usercfg
pycortex_config_file_new = pycortex_config_file[:-4] + '_new.cfg'
pycortex_config_file_old = pycortex_config_file[:-4] + '_old.cfg'

Path(pycortex_config_file_new).touch()
with open(pycortex_config_file, 'r') as fileIn:
    with open(pycortex_config_file_new, 'w') as fileOut:
        for line in fileIn:
            if 'filestore' in line:
                newline = 'filestore=' + pycortex_db
                fileOut.write(newline)
                newline = '\n'
            elif 'colormaps' in line:
                newline = 'colormaps=' + pycortex_cm
                fileOut.write(newline)
                newline = '\n'
            else:
                newline = line
            fileOut.write(newline)
os.rename(pycortex_config_file,pycortex_config_file_old)
os.rename(pycortex_config_file_new, pycortex_config_file)

In [3]:
for subject in subjects:
    
    # Define and create folder
    mask_dir = '{}/{}/masks'.format(pp_dir, subject)
    try: os.makedirs(mask_dir) 
    except: pass

    # Create ROI masks
    # ----------------
    ref_file = "{}/db/{}/transforms/{}/reference.nii.gz".format(pycortex_dir, subject, xfm_name)
    print (ref_file)
    ref_img = nb.load(ref_file)

    for roi in rois:
        roi_mask_file_L = "{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)
        if os.path.isfile(roi_mask_file_L) == 0:
            print('creating {} {} mask'.format(roi, cortical_mask))
            roi_mask = cortex.utils.get_roi_masks(subject=subject, xfmname=xfm_name, gm_sampler=cortical_mask, roi_list=roi, return_dict=True, split_lr=True)
            for hemi in ['R','L']:
                roi_mask_file = "{}/{}_{}_{}.nii.gz".format(mask_dir, roi, cortical_mask, hemi)
                roi_mask_img = nb.Nifti1Image(dataobj=roi_mask['{}_{}'.format(roi, hemi)].transpose((2,1,0)), affine=ref_img.affine, header=ref_img.header)
                roi_mask_img.to_filename(roi_mask_file)
        else:
            print('{} {} masks: already exist'.format(roi, cortical_mask))
            continue

/home/mchevillard/disks/meso_shared/gaze_exp//derivatives/pp_data/cortex/db/sub-001/transforms/prf.fmriprep/reference.nii.gz
creating V1 cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating V2 cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating V3 cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating V3AB cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating LO cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating VO cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating hMT+ cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating iIPS cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating sIPS cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating iPCS cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating sPCS cortical mask
Cutting 0 overlapping voxels (should be < ~50)
creating mPCS cortical mask
Cutting 0 overlapping voxels (sh